In [95]:
import numpy as np
import pandas as pd 
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd
import osmnx as ox
import itertools as it
%matplotlib inline

In [96]:
flow = 10

In [209]:
G = nx.MultiDiGraph()
travel_time_awal = [10,20,25]
key = G.add_edge(1,2, capacity = 2)
key2 = G.add_edge(1,2, capacity = 4)
key3 = G.add_edge(1,2, capacity = 3)


In [210]:
df = nx.to_pandas_edgelist(G)
df['travel_time_awal'] = travel_time_awal
df['travel_time'] = travel_time_awal
df['flow'] = [0,0,0]
df
# edge_attr = df_attributes_only.set_index('id').to_dict('index')

,source,target,capacity,travel_time_awal,travel_time,flow
0,1,2,2,10,10,0
1,1,2,4,20,20,0
2,1,2,3,25,25,0


In [176]:
# nx.set_edge_attributes(G, values = df['travel_time'].values, name="travel_time")
G = nx.from_pandas_edgelist(df, source='source', target='target', edge_attr=True, create_using=nx.MultiDiGraph())
G[1][2]

AtlasView({0: {'capacity': 2, 'travel_time_awal': 10, 'travel_time': 10, 'flow': 0}, 1: {'capacity': 4, 'travel_time_awal': 20, 'travel_time': 20, 'flow': 0}, 2: {'capacity': 3, 'travel_time_awal': 25, 'travel_time': 25, 'flow': 0}})

In [131]:
routes = nx.shortest_path(G, source=1, target=2, weight='travel_time')
routes

[1, 2]

In [101]:
minimum = G[1][2][0]['travel_time']
keys = 0
for index, i in enumerate(G[1][2]):
    if G[1][2][i]['travel_time'] < minimum:
        minimum = G[1][2][i]
        keys = index

keys

0

In [418]:
def routes_jalan(G, routes, iterasi):
    if iterasi == 1:
        t = 'travel_time_awal'
    else:
        t = 'travel_time' + str(iterasi-1)
    list_jalan = []
    for i in range(len(routes)-1):
        minimum = G[routes[i]][routes[i+1]][0][t]
        n = 0
        keys = 0
        for node1, node2, data in G.edges(data=True):
            if data[t] < minimum:
                minimum = data[t]
                keys = n
            n += 1
        list_jalan.append(keys)
    
    return list_jalan

In [103]:
rute_jalan_key = routes_jalan(G, routes)
rute_jalan_key

[0]

## Updating Flow

In [104]:
nx.set_edge_attributes(G, values = 0, name='auxflow')

In [393]:
def updating_mainflow(G, routes, iterasi):

    rute_jalan_key = routes_jalan(G, routes, iterasi)

    for i in range(len(rute_jalan_key)):
        attrs = {(routes[i], routes[i+1], rute_jalan_key[i]): {"flow"+str(iterasi+1): flow}}
        nx.set_edge_attributes(G, attrs)

    return G  

In [435]:
def updating_auxflow(G, routes,iterasi):

    rute_jalan_key = routes_jalan(G, routes, iterasi+1)

    nx.set_edge_attributes(G, 0, 'auxflow'+str(iterasi))

    for i in range(len(rute_jalan_key)):
        attrs = {(routes[i], routes[i+1], rute_jalan_key[i]): {"auxflow"+str(iterasi): flow}}
        nx.set_edge_attributes(G, attrs)

    return G  

In [107]:
nx.to_pandas_edgelist(G)

,source,target,flow,travel_time,travel_time_awal,capacity,auxflow
0,1,2,0,10,10,2,0
1,1,2,0,20,20,4,0
2,1,2,0,25,25,3,0


In [108]:
Gupdate = updating_mainflow(G, routes)
df = nx.to_pandas_edgelist(Gupdate)
df

,source,target,flow,travel_time,travel_time_awal,capacity,auxflow
0,1,2,10,10,10,2,0
1,1,2,0,20,20,4,0
2,1,2,0,25,25,3,0


In [224]:
#This is a link performance function derived from Indonesia's Highway Manual (MKJI 1997).
def lpr_idn(row):
    if np.isnan(row['flow']):
        return row['travel_time']
    else:
        try:
            return row['travel_time_awal']*(1+0.15*(row['flow']/row['capacity'])**4)
        except ValueError:
            return row['travel_time']

In [110]:
list_edge = []
for i in G.edges():
    list_edge.append(i)

for index, i in enumerate(list_edge):
    attrib = {(i[0], i[1], index): {"travel_time": lpr_idn(G[i[0]][i[1]][index])}}
    nx.set_edge_attributes(G, attrib)

nx.to_pandas_edgelist(G)

,source,target,flow,travel_time,travel_time_awal,capacity,auxflow
0,1,2,10,947.5,10,2,0
1,1,2,0,20.0,20,4,0
2,1,2,0,25.0,25,3,0


In [111]:
routes = nx.shortest_path(G, source=1, target=2, weight='travel_time')
routes

[1, 2]

In [112]:
rute_jalan_key = routes_jalan(G, routes)
rute_jalan_key

[1]

In [113]:
Gupdate = updating_auxflow(G, routes)
nx.to_pandas_edgelist(Gupdate)

,source,target,flow,travel_time,travel_time_awal,capacity,auxflow
0,1,2,10,947.5,10,2,0
1,1,2,0,20.0,20,4,10
2,1,2,0,25.0,25,3,0


## Bisection Method

In [385]:
def line_search(row, iterasi, a, alpha, beta):
    t=row['travel_time_awal']
    x=row['flow'+str(iterasi)]
    y=row['auxflow'+str(iterasi)]
    c=row['capacity']
    return -t*(x-y)*(alpha*(((x+a*(y-x))/c)**beta)+1)

In [361]:
def update(row, alpha, iterasi):
    return row['flow'+str(iterasi)] + alpha * (row['auxflow'+str(iterasi)] - row['flow'+str(iterasi)])

In [386]:
def bisection(G, iterasi, xl=0, xr=1, delta=0.0002, alpha=0.15, beta=4):
    n = 0
    df = nx.to_pandas_edgelist(G)
    condition = True
    while condition:
        n += 1
        x = (xl+xr)/2
        if df.apply(lambda row: line_search(row, iterasi, x, alpha=alpha, beta=beta), axis = 1).sum() <= 0:
            xl = x
        else:
            xr = x
        condition = abs(xr-xl) > 2*delta
        print('|xl-xr| =', abs(xr-xl),' xl =', xl, ' xr=',xr)
    print(f"number of iteration in bisection method is {n} with alpha = {(xr+xl)/2}")
    return (xr+xl)/2

In [117]:
nx.to_pandas_edgelist(G)

,source,target,flow,travel_time,travel_time_awal,capacity,auxflow
0,1,2,10,947.5,10,2,0
1,1,2,0,20.0,20,4,10
2,1,2,0,25.0,25,3,0


In [118]:
bisection(G)

|xl-xr| = 0.5  xl = 0.5  xr= 1
|xl-xr| = 0.25  xl = 0.5  xr= 0.75
|xl-xr| = 0.125  xl = 0.5  xr= 0.625
|xl-xr| = 0.0625  xl = 0.5625  xr= 0.625
|xl-xr| = 0.03125  xl = 0.59375  xr= 0.625
|xl-xr| = 0.015625  xl = 0.59375  xr= 0.609375
|xl-xr| = 0.0078125  xl = 0.59375  xr= 0.6015625
|xl-xr| = 0.00390625  xl = 0.59375  xr= 0.59765625
|xl-xr| = 0.001953125  xl = 0.595703125  xr= 0.59765625
|xl-xr| = 0.0009765625  xl = 0.595703125  xr= 0.5966796875
|xl-xr| = 0.00048828125  xl = 0.59619140625  xr= 0.5966796875
|xl-xr| = 0.000244140625  xl = 0.596435546875  xr= 0.5966796875
number of iteration in bisection method is 12


0.5965576171875

In [126]:
alpha = bisection(G)
df = nx.to_pandas_edgelist(G)
tes = df.apply(lambda row: update(row, alpha), axis=1)

for index, i in enumerate(list_edge):
    new_flow = 'flow' + str(2)
    attrib = {(i[0], i[1], index): {new_flow: tes[index]}}
    nx.set_edge_attributes(G, attrib)

nx.to_pandas_edgelist(G)

|xl-xr| = 0.5  xl = 0.5  xr= 1
|xl-xr| = 0.25  xl = 0.5  xr= 0.75
|xl-xr| = 0.125  xl = 0.5  xr= 0.625
|xl-xr| = 0.0625  xl = 0.5625  xr= 0.625
|xl-xr| = 0.03125  xl = 0.59375  xr= 0.625
|xl-xr| = 0.015625  xl = 0.59375  xr= 0.609375
|xl-xr| = 0.0078125  xl = 0.59375  xr= 0.6015625
|xl-xr| = 0.00390625  xl = 0.59375  xr= 0.59765625
|xl-xr| = 0.001953125  xl = 0.595703125  xr= 0.59765625
|xl-xr| = 0.0009765625  xl = 0.595703125  xr= 0.5966796875
|xl-xr| = 0.00048828125  xl = 0.59619140625  xr= 0.5966796875
|xl-xr| = 0.000244140625  xl = 0.596435546875  xr= 0.5966796875
number of iteration in bisection method is 12


,source,target,flow,travel_time,travel_time_awal,capacity,flow2,auxflow
0,1,2,10,947.5,10,2,4.034424,0
1,1,2,0,20.0,20,4,5.965576,10
2,1,2,0,25.0,25,3,0.000000,0


# Convev Combinations Algorithm

In [296]:
#This is a link performance function derived from Indonesia's Highway Manual (MKJI 1997).
def lpr_idn(row, iterasi):
    flow = 'flow'+str(iterasi)
    if np.isnan(row['flow' + str(iterasi)]):
        return row['travel_time']
    else:
        try:
            return row['travel_time_awal']*(1+0.15*(row['flow'+str(iterasi)]/row['capacity'])**4)
        except ValueError:
            return row['travel_time']

In [459]:
def CCA(G, source, target):

    # iterasi 0
    iterasi = 1
    routes = nx.shortest_path(G, source, target, weight='travel_time_awal')
    # rute_jalan_key = routes_jalan(G, routes, iterasi)
    G = updating_mainflow(G, routes, iterasi)

    # iterasi 1
    list_edge = []
    for i in G.edges():
        list_edge.append(i)

    for index, i in enumerate(list_edge):
        attrib = {(i[0], i[1], index): {"travel_time1": lpr_idn(G[i[0]][i[1]][index], iterasi)}}
        nx.set_edge_attributes(G, attrib)

    routes = nx.shortest_path(G, source=1, target=2, weight='travel_time1')
    # rute_jalan_key = routes_jalan(G, routes, iterasi)
    G = updating_auxflow(G, routes, iterasi)

    alpha = bisection(G, iterasi)
    df = nx.to_pandas_edgelist(G)
    flow_baru = df.apply(lambda row: update(row, alpha, iterasi), axis=1)

    for index, i in enumerate(list_edge):
        new_flow = 'flow' + str(2)
        attrib = {(i[0], i[1], index): {new_flow: flow_baru[index]}}
        nx.set_edge_attributes(G, attrib)

    # iterasi 2
    iterasi += 1
    routes = nx.shortest_path(G, source, target, weight='travel_time'+str(iterasi-1))
    # rute_jalan_key = routes_jalan(G, routes, iterasi)
    # G = updating_mainflow(G, routes, iterasi-1)

    list_edge = []
    for i in G.edges():
        list_edge.append(i)

    for index, i in enumerate(list_edge):
        attrib = {(i[0], i[1], index): {"travel_time2": lpr_idn(G[i[0]][i[1]][index], iterasi)}}
        nx.set_edge_attributes(G, attrib)

    routes = nx.shortest_path(G, source=1, target=2, weight='travel_time'+str(iterasi))
    # rute_jalan_key = routes_jalan(G, routes, iterasi)
    G = updating_auxflow(G, routes, iterasi)

    alpha = bisection(G, iterasi)
    df = nx.to_pandas_edgelist(G)
    flow_baru = df.apply(lambda row: update(row, alpha, iterasi), axis=1)

    for index, i in enumerate(list_edge):
        new_flow = 'flow' + str(3)
        attrib = {(i[0], i[1], index): {new_flow: flow_baru[index]}}
        nx.set_edge_attributes(G, attrib)

    # iterasi 3
    iterasi += 1
    routes = nx.shortest_path(G, source, target, weight='travel_time'+str(iterasi-1))
    # rute_jalan_key = routes_jalan(G, routes, iterasi)
    # G = updating_mainflow(G, routes, iterasi-1)

    list_edge = []
    for i in G.edges():
        list_edge.append(i)

    for index, i in enumerate(list_edge):
        attrib = {(i[0], i[1], index): {"travel_time3": lpr_idn(G[i[0]][i[1]][index], iterasi)}}
        nx.set_edge_attributes(G, attrib)

    routes = nx.shortest_path(G, source=1, target=2, weight='travel_time'+str(iterasi))
    # rute_jalan_key = routes_jalan(G, routes, iterasi)
    G = updating_auxflow(G, routes, iterasi)

    alpha = bisection(G, iterasi)
    df = nx.to_pandas_edgelist(G)
    flow_baru = df.apply(lambda row: update(row, alpha, iterasi), axis=1)

    for index, i in enumerate(list_edge):
        new_flow = 'flow' + str(4)
        attrib = {(i[0], i[1], index): {new_flow: flow_baru[index]}}
        nx.set_edge_attributes(G, attrib)

    return G


In [460]:
# Contoh
G = nx.MultiDiGraph()
travel_time_awal = [10,20,25]
key = G.add_edge(1,2, capacity = 2)
key2 = G.add_edge(1,2, capacity = 4)
key3 = G.add_edge(1,2, capacity = 3)

df = nx.to_pandas_edgelist(G)
df['travel_time_awal'] = travel_time_awal
df['travel_time1'] = travel_time_awal
df['flow1'] = [0,0,0]
df
# edge_attr = df_attributes_only.set_index('id').to_dict('index')
# nx.set_edge_attributes(G, values = df['travel_time'].values, name="travel_time")
G = nx.from_pandas_edgelist(df, source='source', target='target', edge_attr=True, create_using=nx.MultiDiGraph())

H = CCA(G,1,2)
df = nx.to_pandas_edgelist(H)
df

|xl-xr| = 0.5  xl = 0.5  xr= 1
|xl-xr| = 0.25  xl = 0.5  xr= 0.75
|xl-xr| = 0.125  xl = 0.5  xr= 0.625
|xl-xr| = 0.0625  xl = 0.5625  xr= 0.625
|xl-xr| = 0.03125  xl = 0.59375  xr= 0.625
|xl-xr| = 0.015625  xl = 0.59375  xr= 0.609375
|xl-xr| = 0.0078125  xl = 0.59375  xr= 0.6015625
|xl-xr| = 0.00390625  xl = 0.59375  xr= 0.59765625
|xl-xr| = 0.001953125  xl = 0.595703125  xr= 0.59765625
|xl-xr| = 0.0009765625  xl = 0.595703125  xr= 0.5966796875
|xl-xr| = 0.00048828125  xl = 0.59619140625  xr= 0.5966796875
|xl-xr| = 0.000244140625  xl = 0.596435546875  xr= 0.5966796875
number of iteration in bisection method is 12 with alpha = 0.5965576171875
|xl-xr| = 0.5  xl = 0  xr= 0.5
|xl-xr| = 0.25  xl = 0  xr= 0.25
|xl-xr| = 0.125  xl = 0.125  xr= 0.25
|xl-xr| = 0.0625  xl = 0.125  xr= 0.1875
|xl-xr| = 0.03125  xl = 0.15625  xr= 0.1875
|xl-xr| = 0.015625  xl = 0.15625  xr= 0.171875
|xl-xr| = 0.0078125  xl = 0.15625  xr= 0.1640625
|xl-xr| = 0.00390625  xl = 0.16015625  xr= 0.1640625
|xl-xr| = 0.00

,source,target,flow1,auxflow2,travel_time2,auxflow3,flow4,travel_time_awal,capacity,travel_time1,flow3,travel_time3,flow2,auxflow1
0,1,2,10,0,34.836898,10,3.619825,10,2,947.5,3.384838,22.306171,4.034424,0
1,1,2,0,0,34.841947,0,4.827262,20,4,20.0,5.005054,27.353879,5.965576,10
2,1,2,0,10,25.000000,0,1.552912,25,3,25.0,1.610107,25.311147,0.000000,0


In [266]:
routes = nx.shortest_path(H, 1, 2, weight='travel_time')
rute_jalan_key = routes_jalan(H, routes)
# nx.set_edge_attributes(H, 0, 'travel_time2')

list_edge = []
for i in G.edges():
    list_edge.append(i)

for index, i in enumerate(list_edge):
    attrib = {(i[0], i[1], index): {"travel_time2": lpr_idn(H[i[0]][i[1]][index])}}
    nx.set_edge_attributes(G, attrib)

nx.to_pandas_edgelist(H)

,source,target,flow,travel_time,travel_time_awal,capacity,flow2,auxflow,travel_time2
0,1,2,10,947.5,10,2,4.034424,0,34.836898
1,1,2,0,20.0,20,4,5.965576,10,34.841947
2,1,2,0,25.0,25,3,0.000000,0,25.000000
